# Rhodium: Circular Arithmetic for Geographic Coordinates

This notebook demonstrates how rhodium solves common bugs in geographic coordinate calculations.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marszdf/rhodium/blob/main/rhodium_demo.ipynb)

## Installation

In [ ]:
# Install rhodium and optional dependencies for visualization
!pip install elemental-rhodium matplotlib numpy

## The Problem: Naive Arithmetic Fails at Boundaries

Geographic coordinates wrap around at boundaries (360° for bearings, ±180° for longitude). Regular arithmetic produces wrong results.

In [ ]:
# Example 1: Aircraft heading change
heading_1 = 355  # degrees (almost north)
heading_2 = 5    # degrees (just past north)

# WRONG: Naive arithmetic
naive_diff = heading_2 - heading_1
print(f"❌ Naive difference: {naive_diff}° (WRONG! Says turned 350° left)")

# CORRECT: Using rhodium
from rhodium import bearing
correct_diff = bearing.diff(heading_1, heading_2)
print(f"✅ Rhodium difference: {correct_diff}° (Correct! Turned 10° right)")

In [ ]:
# Example 2: Pacific Ocean longitude averaging
tokyo_lng = 139.7      # Japan
san_francisco_lng = -122.4  # USA

# WRONG: Naive average
naive_avg = (tokyo_lng + san_francisco_lng) / 2
print(f"❌ Naive average: {naive_avg}° (WRONG! That's in the Atlantic Ocean)")

# CORRECT: Using rhodium
from rhodium import lng
correct_avg = lng.mean(tokyo_lng, san_francisco_lng)
print(f"✅ Rhodium average: {correct_avg}° (Correct! In the Pacific Ocean)")

## Visualization 1: Bearing Arithmetic on a Compass Rose

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from rhodium import bearing

def plot_bearing_difference(b1, b2):
    """Visualize bearing difference on a compass rose."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), subplot_kw={'projection': 'polar'})
    
    # Convert to radians for plotting (0° = North = top)
    b1_rad = np.radians(90 - b1)
    b2_rad = np.radians(90 - b2)
    
    # Left plot: Naive arithmetic
    naive_diff = b2 - b1
    ax1.set_theta_zero_location('N')
    ax1.set_theta_direction(-1)
    ax1.arrow(0, 0, b1_rad, 0.8, width=0.05, head_width=0.15, head_length=0.1, fc='blue', ec='blue', label=f'Start: {b1}°')
    ax1.arrow(0, 0, b2_rad, 0.8, width=0.05, head_width=0.15, head_length=0.1, fc='red', ec='red', label=f'End: {b2}°')
    ax1.set_ylim(0, 1)
    ax1.set_title(f'❌ Naive: {naive_diff}° difference\n(WRONG)', fontsize=14, pad=20)
    ax1.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    
    # Right plot: Rhodium arithmetic
    correct_diff = bearing.diff(b1, b2)
    ax2.set_theta_zero_location('N')
    ax2.set_theta_direction(-1)
    ax2.arrow(0, 0, b1_rad, 0.8, width=0.05, head_width=0.15, head_length=0.1, fc='blue', ec='blue', label=f'Start: {b1}°')
    ax2.arrow(0, 0, b2_rad, 0.8, width=0.05, head_width=0.15, head_length=0.1, fc='red', ec='red', label=f'End: {b2}°')
    ax2.set_ylim(0, 1)
    ax2.set_title(f'✅ Rhodium: {correct_diff}° difference\n(Correct!)', fontsize=14, pad=20, color='green')
    ax2.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    
    plt.tight_layout()
    plt.show()

# Demo: Aircraft heading change
plot_bearing_difference(355, 5)

## Visualization 2: Longitude Averaging Across the Antimeridian

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from rhodium import lng

def plot_longitude_average(lng1, lng2, lat1, lat2, city1_name, city2_name):
    """Visualize longitude averaging on a world map."""
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Draw world map outline
    ax.axhline(0, color='gray', linewidth=0.5, linestyle='--', alpha=0.5)
    ax.axvline(0, color='gray', linewidth=0.5, linestyle='--', alpha=0.5)
    ax.axvline(180, color='red', linewidth=2, linestyle='--', alpha=0.7, label='Antimeridian (±180°)')
    ax.axvline(-180, color='red', linewidth=2, linestyle='--', alpha=0.7)
    
    # Plot cities
    ax.plot(lng1, lat1, 'bo', markersize=15, label=f'{city1_name} ({lng1}°, {lat1}°)')
    ax.plot(lng2, lat2, 'go', markersize=15, label=f'{city2_name} ({lng2}°, {lat2}°)')
    
    # Naive average
    naive_avg = (lng1 + lng2) / 2
    naive_lat = (lat1 + lat2) / 2
    ax.plot(naive_avg, naive_lat, 'rx', markersize=20, markeredgewidth=3, label=f'❌ Naive avg: {naive_avg:.1f}°')
    
    # Rhodium average
    correct_avg = lng.mean(lng1, lng2)
    ax.plot(correct_avg, naive_lat, 'g*', markersize=25, markeredgewidth=2, label=f'✅ Rhodium avg: {correct_avg:.1f}°')
    
    # Draw great circle path (simplified)
    if abs(lng1 - lng2) > 180:
        # Crosses antimeridian - draw two segments
        ax.plot([lng1, 180], [lat1, lat1], 'b--', alpha=0.3, linewidth=2)
        ax.plot([-180, lng2], [lat2, lat2], 'b--', alpha=0.3, linewidth=2)
    else:
        ax.plot([lng1, lng2], [lat1, lat2], 'b--', alpha=0.3, linewidth=2)
    
    ax.set_xlim(-180, 180)
    ax.set_ylim(-90, 90)
    ax.set_xlabel('Longitude', fontsize=12)
    ax.set_ylabel('Latitude', fontsize=12)
    ax.set_title('Longitude Averaging: Naive vs Rhodium', fontsize=14, pad=20)
    ax.legend(loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Demo: Tokyo to San Francisco
plot_longitude_average(139.7, -122.4, 35.7, 37.8, 'Tokyo', 'San Francisco')

## Bearing Arithmetic API

All operations handle wraparound correctly at 360°.

In [ ]:
from rhodium import bearing

# Normalize to [0, 360)
print(bearing.normalize(720))    # 0.0
print(bearing.normalize(-45))    # 315.0

# Shortest angular difference
print(bearing.diff(10, 350))     # -20.0 (not 340)
print(bearing.diff(350, 10))     # 20.0 (not -340)

# Average bearing
print(bearing.mean(350, 10))     # 0.0 (crosses north)

# Interpolation
print(bearing.interpolate(350, 10, 0.5))  # 0.0 (halfway)

# Opposite bearing
print(bearing.opposite(45))      # 225.0

# Check if within tolerance
print(bearing.within(5, 355, tolerance=15))  # True

## Longitude Arithmetic API

All operations handle the antimeridian (±180°) correctly.

In [ ]:
from rhodium import lng

# Normalize to (-180, 180]
print(lng.normalize(190))        # -170.0
print(lng.normalize(-190))       # 170.0

# Shortest difference
print(lng.diff(170, -170))       # -20.0 (crosses antimeridian)

# Average (handles Pacific Ocean)
print(lng.mean(170, -170))       # 180.0
print(lng.mean(139.7, -122.4))   # Approximately -171.35 (Pacific)

# Interpolation
print(lng.interpolate(170, -170, 0.5))  # 180.0

## Latitude Operations API

Latitude has hard boundaries at ±90°.

In [ ]:
from rhodium import lat

# Clamp to [-90, 90]
print(lat.clamp(100))            # 90.0
print(lat.clamp(-100))           # -90.0

# Validation
print(lat.is_valid(45))          # True
print(lat.is_valid(100))         # False

# Midpoint
print(lat.midpoint(10, 50))      # 30.0

# Hemisphere
print(lat.hemisphere(45))        # 'N'
print(lat.hemisphere(-33.9))     # 'S'

# Check if within range
print(lat.within(45, 40, 50))    # True

## Bounding Boxes: The Alaska/Fiji Problem

Regular bounding boxes fail when crossing the antimeridian.

In [ ]:
from rhodium import bbox
from rhodium.bbox import Point, BBox

# Fiji crosses the antimeridian (west=177°, east=-178°)
fiji = BBox(west=177, east=-178, south=-19, north=-16)

print(f"Fiji width: {bbox.width(fiji)}° (not negative!)")
print(f"Fiji height: {bbox.height(fiji)}°")
print(f"Crosses antimeridian: {bbox.crosses_antimeridian(fiji)}")

# Check if point is in Fiji
suva = Point(lng=178.4, lat=-18.1)  # Suva, Fiji's capital
print(f"Suva in Fiji bbox: {bbox.contains(fiji, suva)}")

## Visualization 3: Bounding Box with Antimeridian Crossing

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from rhodium import bbox
from rhodium.bbox import BBox, Point

def plot_bbox(box, points=None, title="Bounding Box"):
    """Visualize a bounding box on a map."""
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Draw world map outline
    ax.axhline(0, color='gray', linewidth=0.5, linestyle='--', alpha=0.5)
    ax.axvline(0, color='gray', linewidth=0.5, linestyle='--', alpha=0.5)
    ax.axvline(180, color='red', linewidth=2, linestyle='--', alpha=0.7, label='Antimeridian')
    ax.axvline(-180, color='red', linewidth=2, linestyle='--', alpha=0.7)
    
    # Draw bounding box
    if bbox.crosses_antimeridian(box):
        # Draw two rectangles for antimeridian-crossing box
        # Left rectangle (from west to 180)
        width1 = 180 - box.west
        rect1 = patches.Rectangle(
            (box.west, box.south), width1, box.north - box.south,
            linewidth=3, edgecolor='blue', facecolor='lightblue', alpha=0.3
        )
        ax.add_patch(rect1)
        
        # Right rectangle (from -180 to east)
        width2 = box.east - (-180)
        rect2 = patches.Rectangle(
            (-180, box.south), width2, box.north - box.south,
            linewidth=3, edgecolor='blue', facecolor='lightblue', alpha=0.3,
            label=f'BBox: ({box.west}°, {box.east}°, {box.south}°, {box.north}°)'
        )
        ax.add_patch(rect2)
    else:
        # Regular rectangle
        width = box.east - box.west
        rect = patches.Rectangle(
            (box.west, box.south), width, box.north - box.south,
            linewidth=3, edgecolor='blue', facecolor='lightblue', alpha=0.3,
            label=f'BBox: ({box.west}°, {box.east}°, {box.south}°, {box.north}°)'
        )
        ax.add_patch(rect)
    
    # Plot points if provided
    if points:
        for point, name in points:
            is_inside = bbox.contains(box, point)
            color = 'green' if is_inside else 'red'
            marker = 'o' if is_inside else 'x'
            ax.plot(point.lng, point.lat, marker, markersize=15, 
                   color=color, markeredgewidth=2,
                   label=f'{name}: {"✅ inside" if is_inside else "❌ outside"}')
    
    # Plot center
    center = bbox.center(box)
    ax.plot(center.lng, center.lat, '*', markersize=20, color='orange', 
           markeredgewidth=2, markeredgecolor='black', label=f'Center: ({center.lng:.1f}°, {center.lat:.1f}°)')
    
    ax.set_xlim(-180, 180)
    ax.set_ylim(-90, 90)
    ax.set_xlabel('Longitude', fontsize=12)
    ax.set_ylabel('Latitude', fontsize=12)
    ax.set_title(title, fontsize=14, pad=20)
    ax.legend(loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Demo: Fiji (crosses antimeridian)
fiji = BBox(west=177, east=-178, south=-19, north=-16)
suva = Point(lng=178.4, lat=-18.1)
auckland = Point(lng=174.8, lat=-36.8)  # Outside Fiji

plot_bbox(fiji, [(suva, 'Suva'), (auckland, 'Auckland')], 
         f"Fiji Bounding Box (crosses antimeridian)\nWidth: {bbox.width(fiji):.1f}°, Height: {bbox.height(fiji):.1f}°")

## Bounding Box API

In [ ]:
from rhodium import bbox
from rhodium.bbox import Point, BBox

# Create from corner points
sw = Point(lng=-122.5, lat=37.7)
ne = Point(lng=-122.3, lat=37.9)
sf_box = bbox.create(sw, ne)

# Create from list of points
points = [
    Point(lng=177, lat=-18),
    Point(lng=-178, lat=-17),
    Point(lng=179, lat=-19)
]
fiji_box = bbox.from_points(points)

# Dimensions
print(f"Width: {bbox.width(fiji_box)}°")
print(f"Height: {bbox.height(fiji_box)}°")
print(f"Center: {bbox.center(fiji_box)}")

# Point containment
point = Point(lng=178, lat=-18)
print(f"Contains point: {bbox.contains(fiji_box, point)}")

# Box operations
box1 = BBox(west=0, east=10, south=0, north=10)
box2 = BBox(west=5, east=15, south=5, north=15)

print(f"Boxes intersect: {bbox.intersects(box1, box2)}")
intersection = bbox.intersection(box1, box2)
union = bbox.union(box1, box2)

# Expand box to include a point
expanded = bbox.expand(box1, Point(lng=20, lat=20))
print(f"Expanded box: {expanded}")

## Pythonic Property Access

BBox provides convenient properties alongside the functional API.

In [ ]:
from rhodium.bbox import BBox

fiji = BBox(west=177, east=-178, south=-19, north=-16)

# Functional API
from rhodium import bbox
print(f"Width (functional): {bbox.width(fiji)}°")

# Property API (same result)
print(f"Width (property): {fiji.width}°")
print(f"Height: {fiji.height}°")
print(f"Center: {fiji.center_point}")
print(f"Crosses antimeridian: {fiji.crosses_antimeridian}")

## GeoJSON Integration via `__geo_interface__`

Rhodium implements the Python `__geo_interface__` protocol for compatibility with shapely, fiona, geopandas, and other GIS libraries.

In [ ]:
from rhodium.bbox import Point, BBox
import json

# Point to GeoJSON
point = Point(lng=-122.4, lat=37.8)
print("Point GeoJSON:")
print(json.dumps(point.__geo_interface__, indent=2))

# BBox to GeoJSON Polygon
box = BBox(west=0, east=10, south=0, north=10)
print("\nBBox GeoJSON:")
print(json.dumps(box.__geo_interface__, indent=2))

## Integration with Shapely (Optional)

If you have shapely installed, rhodium objects work seamlessly with it.

In [ ]:
# This cell requires shapely - uncomment to try
# !pip install shapely

# from shapely.geometry import shape
# from rhodium.bbox import Point, BBox

# # Convert rhodium Point to shapely Point
# rh_point = Point(lng=-122.4, lat=37.8)
# sh_point = shape(rh_point.__geo_interface__)
# print(f"Shapely Point: {sh_point}")

# # Convert rhodium BBox to shapely Polygon
# rh_box = BBox(west=0, east=10, south=0, north=10)
# sh_polygon = shape(rh_box.__geo_interface__)
# print(f"Shapely Polygon area: {sh_polygon.area}")

## Performance Characteristics

Rhodium is optimized for correctness, not bulk processing.

In [ ]:
import time
from rhodium import bearing, lng, lat, bbox
from rhodium.bbox import Point, BBox

# Benchmark bearing operations
iterations = 100_000

start = time.perf_counter()
for _ in range(iterations):
    bearing.normalize(365.5)
elapsed = time.perf_counter() - start
print(f"bearing.normalize(): {elapsed/iterations*1e6:.2f} µs per operation ({iterations/elapsed:,.0f} ops/sec)")

start = time.perf_counter()
for _ in range(iterations):
    bearing.diff(350, 10)
elapsed = time.perf_counter() - start
print(f"bearing.diff(): {elapsed/iterations*1e6:.2f} µs per operation ({iterations/elapsed:,.0f} ops/sec)")

# Benchmark bbox operations
box = BBox(west=0, east=10, south=0, north=10)
point = Point(lng=5, lat=5)

start = time.perf_counter()
for _ in range(iterations):
    bbox.contains(box, point)
elapsed = time.perf_counter() - start
print(f"bbox.contains(): {elapsed/iterations*1e6:.2f} µs per operation ({iterations/elapsed:,.0f} ops/sec)")

## Error Handling

Rhodium validates inputs and provides clear error messages.

In [ ]:
from rhodium.bbox import Point, BBox
from rhodium import InvalidLatitudeError, InvalidLongitudeError, InvalidBBoxError

# Invalid latitude
try:
    Point(lng=0, lat=100)  # Latitude must be in [-90, 90]
except InvalidLatitudeError as e:
    print(f"❌ {e}")

# Invalid longitude
try:
    Point(lng=200, lat=0)  # Longitude must be in [-180, 180]
except InvalidLongitudeError as e:
    print(f"❌ {e}")

# Invalid bounding box
try:
    BBox(west=0, east=10, south=50, north=10)  # South cannot be > North
except InvalidBBoxError as e:
    print(f"❌ {e}")

## Real-World Use Cases

### 1. Aviation: Flight Path Calculations

In [ ]:
from rhodium import bearing

# Aircraft turning from 355° to 5° (crossing north)
current_heading = 355
target_heading = 5

turn_angle = bearing.diff(current_heading, target_heading)
print(f"Turn angle: {turn_angle}°")
print(f"Direction: {'right' if turn_angle > 0 else 'left'}")
print(f"Magnitude: {abs(turn_angle)}°")

### 2. Maritime: Pacific Ocean Routes

In [ ]:
from rhodium import lng

# Ship route from Tokyo to Los Angeles
tokyo_lng = 139.7
la_lng = -118.2

# Midpoint for refueling stop
midpoint = lng.mean(tokyo_lng, la_lng)
print(f"Refueling point longitude: {midpoint:.1f}° (in Pacific Ocean)")

# Total distance in longitude degrees
distance = abs(lng.diff(tokyo_lng, la_lng))
print(f"Longitude distance: {distance:.1f}°")

### 3. GIS: Bounding Box for Aleutian Islands

In [ ]:
from rhodium import bbox
from rhodium.bbox import Point, BBox

# Aleutian Islands span the antimeridian
aleutians = BBox(west=165, east=-163, south=51, north=55)

# Check if points are in the Aleutians
attu_island = Point(lng=173.0, lat=52.9)  # Westernmost point of Alaska
unalaska = Point(lng=-166.5, lat=53.9)    # Dutch Harbor

print(f"Attu Island in bbox: {bbox.contains(aleutians, attu_island)}")
print(f"Unalaska in bbox: {bbox.contains(aleutians, unalaska)}")
print(f"Bbox width: {bbox.width(aleutians):.1f}°")
print(f"Bbox center: {bbox.center(aleutians)}")

## Learn More

- **GitHub**: [github.com/marszdf/rhodium](https://github.com/marszdf/rhodium)
- **PyPI**: `pip install elemental-rhodium`
- **Documentation**: See README.md for full API reference
- **Issues/Questions**: [GitHub Issues](https://github.com/marszdf/rhodium/issues)

## Summary

✅ **Use rhodium when**:
- Working with bearings, headings, or compass directions
- Handling longitude coordinates near ±180° (Pacific Ocean, Arctic, etc.)
- Creating bounding boxes that cross the antimeridian
- Averaging or interpolating circular quantities
- Integrating with Python GIS tools (shapely, geopandas)

❌ **Don't use rhodium for**:
- Geodesic distance calculations (use `pyproj` or `geopy`)
- Point-in-polygon tests (use `shapely`)
- Bulk processing 100k+ coordinates (use `numpy` + custom logic)
- Coordinate system transformations (use `pyproj`)